In [ ]:
import os
import kagglehub

# --- Définition des chemins ---
# Le dossier principal qui contiendra les données
DATA_DIR = "data"
# Le dossier de destination spécifique au téléchargement Kaggle
CSV_DIR = os.path.join(DATA_DIR, "csv")
# Le handle Kaggle pour le dataset
KAGGLE_HANDLE = "afrniomelo/tep-csv"

# --- 1. Vérification et Création des Dossiers ---
if not os.path.exists(DATA_DIR):
    # !mkdir data
    os.makedirs(DATA_DIR, exist_ok=True)
    print(f"Dossier créé: {DATA_DIR}")

if not os.path.exists(CSV_DIR):
    # !mkdir data/csv
    os.makedirs(CSV_DIR, exist_ok=True)
    print(f"Dossier créé: {CSV_DIR}")


# --- 2. Vérification du Contenu avant le Téléchargement ---
# On vérifie si le dossier de destination contient déjà des fichiers (autres que les fichiers cachés/système)
existing_files = [f for f in os.listdir(CSV_DIR) if not f.startswith('.')]

if existing_files:
    print(f"✅ Le dossier {CSV_DIR} contient déjà {len(existing_files)} fichier(s).")
    print("Le téléchargement sera ignoré.")

    # Optional: Si vous devez obtenir le chemin d'accès au dossier pour la suite du script,
    # vous pouvez le définir ici.
    path = CSV_DIR

else:
    print(f"❌ Le dossier {CSV_DIR} est vide. Démarrage du téléchargement...")

    # Download latest version
    # Note: L'argument 'path' de kagglehub.dataset_download DOIT être le chemin
    # où le dataset sera téléchargé (CSV_DIR dans ce cas)
    try:
        path = kagglehub.dataset_download(handle=KAGGLE_HANDLE, path=CSV_DIR)
        print("Téléchargement réussi!")
    except Exception as e:
        print(f"Erreur lors du téléchargement Kaggle : {e}")
        path = None


# --- 3. Affichage du Résultat ---
if path:
    print("Chemin d'accès aux fichiers du dataset:", os.path.abspath(path))

✅ Le dossier data/csv contient déjà 8 fichier(s).
Le téléchargement sera ignoré.
Chemin d'accès aux fichiers du dataset: /home/bapt/code/Monitor-the-Reactor/Explo/data/csv


In [ ]:
import pandas as pd
import os
from typing import Iterator, Generator, List, Dict

# --- Constantes ---
CSV_FILE_PATH = None
CHUNK_SIZE = 100000  # Définir une taille de morceau (par exemple, 100 000 lignes)

def process_csv_by_chunks(path: str, chunk_size: int) -> List[pd.DataFrame] | None:
    """
    Charge un fichier CSV par morceaux, traite chaque morceau,
    et retourne une liste des DataFrames traités.

    Args:
        path (str): Le chemin vers le fichier CSV.
        chunk_size (int): Le nombre de lignes à lire à la fois.

    Returns:
        List[pd.DataFrame] | None: Une liste des DataFrames traités, ou None en cas d'erreur.
    """

    print(f"Tentative de chargement du fichier : {os.path.abspath(path)}")
    print(f"Chargement par morceaux de taille : {chunk_size} lignes.")

    if not os.path.exists(path):
        print(f"Erreur: Le fichier '{path}' est introuvable. Vérifiez le chemin d'accès.")
        return None

    processed_chunks = []
    chunk_index = 0

    try:
        # Créer un itérateur (TextFileReader) au lieu d'un DataFrame unique
        csv_iterator = pd.read_csv(path, chunksize=chunk_size)

        # Parcourir les morceaux générés par l'itérateur
        for chunk in csv_iterator:
            chunk_index += 1
            print(f"Traitement du morceau #{chunk_index} (taille: {len(chunk)} lignes)...")

            # 💡 --- Zone de Traitement des Données --- 💡
            # Ici, vous pouvez appliquer des opérations qui réduisent la taille du morceau,
            # comme le filtrage, l'agrégation ou le calcul de statistiques.

            # Exemple : Calculer la moyenne de toutes les colonnes et stocker
            # stats_df = chunk.mean().to_frame().T
            # processed_chunks.append(stats_df)

            # Exemple : Filtrer pour garder uniquement les lignes où 'col_A' > 10
            # filtered_chunk = chunk[chunk['col_A'] > 10]
            # processed_chunks.append(filtered_chunk)

            # --- Fin de la Zone de Traitement ---

            # Dans cet exemple, nous stockons le morceau complet filtré
            # Si vous avez 500Mo, vous DEVEZ faire un traitement pour réduire le morceau avant de l'ajouter
            # à 'processed_chunks', sinon vous resaturerez votre RAM.
            processed_chunks.append(chunk)


        print(f"\nChargement et traitement terminés. {chunk_index} morceaux traités.")

        # ⚠️ ATTENTION : La ligne suivante va CONSOLIDER TOUS les morceaux.
        # Si vous n'avez pas réduit la taille des morceaux, vous risquez une saturation RAM.
        # Si vous n'avez besoin que de statistiques, vous pouvez retourner processed_chunks directement.
        final_dataframe = pd.concat(processed_chunks, ignore_index=True)
        print(f"Taille du DataFrame final: {len(final_dataframe)} lignes.")
        return final_dataframe

    except Exception as e:
        print(f"Une erreur s'est produite lors du traitement du fichier CSV : {e}")
        return None

In [1]:
import pandas as pd
import os
from typing import List

# --- Constantes ---
CHUNK_SIZE = 100000  # Taille de morceau


def extract_rows_conditionally(
    path: str,
    chunk_size: int,
    condition: bool,
    window_size: int = 41,
    export_path: str = None
) -> pd.DataFrame | None:
    """
    Charge un fichier CSV par morceaux et extrait des lignes spécifiques
    selon une condition booléenne (toutes les colonnes conservées).

    Args:
        path (str): Le chemin vers le fichier CSV.
        chunk_size (int): Le nombre de lignes à lire à la fois.
        condition (bool): Si True, extrait depuis 0. Si False, extrait depuis 140.
        window_size (int): Taille de la fenêtre (41 ou 81 lignes). Par défaut 41.
        export_path (str): Chemin où exporter le DataFrame en CSV. Si None, pas d'export.

    Returns:
        pd.DataFrame | None: Un DataFrame contenant les lignes extraites, ou None en cas d'erreur.
    """

    print(f"Tentative de chargement du fichier : {os.path.abspath(path)}")
    print(f"Chargement par morceaux de taille : {chunk_size} lignes.")
    print(f"Taille de fenêtre : {window_size} lignes")

    if condition:
        if window_size == 41:
            print("Condition = True, fenêtre 41 → Extraction des lignes 0 à 40")
            start_idx, end_idx = 0, 41
        else:  # window_size == 81
            print("Condition = True, fenêtre 81 → Extraction des lignes 0 à 80")
            start_idx, end_idx = 0, 81
    else:
        if window_size == 41:
            print("Condition = False, fenêtre 41 → Extraction des lignes 140 à 180")
            start_idx, end_idx = 140, 181
        else:  # window_size == 81
            print("Condition = False, fenêtre 81 → Extraction des lignes 140 à 220")
            start_idx, end_idx = 140, 221

    if not os.path.exists(path):
        print(f"Erreur: Le fichier '{path}' est introuvable. Vérifiez le chemin d'accès.")
        return None

    extracted_chunks = []
    chunk_index = 0

    try:
        # Créer un itérateur pour lire le CSV par morceaux
        csv_iterator = pd.read_csv(path, chunksize=chunk_size)

        # Parcourir les morceaux
        for chunk in csv_iterator:
            chunk_index += 1
            print(f"Traitement du morceau #{chunk_index} (taille: {len(chunk)} lignes)...")

            # Déterminer les indices valides pour ce chunk
            max_available = len(chunk)
            actual_start = min(start_idx, max_available)
            actual_end = min(end_idx, max_available)

            if actual_start < actual_end:
                # Extraire les lignes demandées (toutes les colonnes)
                extracted_slice = chunk.iloc[actual_start:actual_end, :]
                extracted_chunks.append(extracted_slice)
                print(f"  → Extrait {len(extracted_slice)} lignes (indices {actual_start} à {actual_end-1})")
            else:
                print(f"  → Pas assez de données dans ce chunk (seulement {max_available} lignes)")

        if not extracted_chunks:
            print("Aucune ligne extraite.")
            return None

        # Concaténer tous les morceaux extraits
        final_df = pd.concat(extracted_chunks, ignore_index=True)
        print(f"\nExtraction terminée. Total: {len(final_df)} lignes extraites avec {final_df.shape[1]} colonnes.")

        # Exporter si un chemin est spécifié
        if export_path is not None:
            try:
                final_df.to_csv(export_path, index=False)
                print(f"✓ DataFrame exporté avec succès vers : {os.path.abspath(export_path)}")
            except Exception as e:
                print(f"✗ Erreur lors de l'export : {e}")

        return final_df

    except Exception as e:
        print(f"Une erreur s'est produite lors du traitement du fichier CSV : {e}")
        return None


# --- Exemple d'utilisation ---
if __name__ == "__main__":
    # Exemple avec un CSV fictif
    csv_path = "votre_fichier.csv"

    # Cas 1: Extraire lignes 0-40 (fenêtre 41) et exporter
    resultat_debut = extract_rows_conditionally(
        path=csv_path,
        chunk_size=CHUNK_SIZE,
        condition=True,
        window_size=41,
        export_path="resultat_debut.csv"
    )

    if resultat_debut is not None:
        print(f"\nRésultat (condition=True, fenêtre=41):")
        print(f"Shape: {resultat_debut.shape}")
        print(f"\nPremières lignes:\n{resultat_debut.head()}")

    print("\n" + "="*60 + "\n")

    # Cas 2: Extraire lignes 140-220 (fenêtre 81) sans export
    resultat_milieu = extract_rows_conditionally(
        path=csv_path,
        chunk_size=CHUNK_SIZE,
        condition=False,
        window_size=81,
        export_path=None  # Pas d'export
    )

    if resultat_milieu is not None:
        print(f"\nRésultat (condition=False, fenêtre=81):")
        print(f"Shape: {resultat_milieu.shape}")
        print(f"\nPremières lignes:\n{resultat_milieu.head()}")

    print("\n" + "="*60 + "\n")

    # Cas 3: Extraire lignes 0-80 (fenêtre 81) avec export
    resultat_large = extract_rows_conditionally(
        path=csv_path,
        chunk_size=CHUNK_SIZE,
        condition=True,
        window_size=81,
        export_path="resultat_large.csv"
    )

    if resultat_large is not None:
        print(f"\nRésultat (condition=True, fenêtre=81):")
        print(f"Shape: {resultat_large.shape}")
        print(f"\nPremières lignes:\n{resultat_large.head()}")

Tentative de chargement du fichier : /home/bapt/code/Monitor-the-Reactor/Explo/votre_fichier.csv
Chargement par morceaux de taille : 100000 lignes.
Taille de fenêtre : 41 lignes
Condition = True, fenêtre 41 → Extraction des lignes 0 à 40
Erreur: Le fichier 'votre_fichier.csv' est introuvable. Vérifiez le chemin d'accès.


Tentative de chargement du fichier : /home/bapt/code/Monitor-the-Reactor/Explo/votre_fichier.csv
Chargement par morceaux de taille : 100000 lignes.
Taille de fenêtre : 81 lignes
Condition = False, fenêtre 81 → Extraction des lignes 140 à 220
Erreur: Le fichier 'votre_fichier.csv' est introuvable. Vérifiez le chemin d'accès.


Tentative de chargement du fichier : /home/bapt/code/Monitor-the-Reactor/Explo/votre_fichier.csv
Chargement par morceaux de taille : 100000 lignes.
Taille de fenêtre : 81 lignes
Condition = True, fenêtre 81 → Extraction des lignes 0 à 80
Erreur: Le fichier 'votre_fichier.csv' est introuvable. Vérifiez le chemin d'accès.


In [3]:
extract_rows_conditionally(path='/home/bapt/code/Monitor-the-Reactor/raw_data/TEP_FaultFree_Testing.csv',
                           chunk_size=100000,
                           condition=True, export_path='/home/bapt/code/Monitor-the-Reactor/raw_data/data_2h')

Tentative de chargement du fichier : /home/bapt/code/Monitor-the-Reactor/raw_data/TEP_FaultFree_Testing.csv
Chargement par morceaux de taille : 100000 lignes.
Taille de fenêtre : 41 lignes
Condition = True, fenêtre 41 → Extraction des lignes 0 à 40
Traitement du morceau #1 (taille: 100000 lignes)...
  → Extrait 41 lignes (indices 0 à 40)
Traitement du morceau #2 (taille: 100000 lignes)...
  → Extrait 41 lignes (indices 0 à 40)
Traitement du morceau #3 (taille: 100000 lignes)...
  → Extrait 41 lignes (indices 0 à 40)
Traitement du morceau #4 (taille: 100000 lignes)...
  → Extrait 41 lignes (indices 0 à 40)
Traitement du morceau #5 (taille: 80000 lignes)...
  → Extrait 41 lignes (indices 0 à 40)

Extraction terminée. Total: 205 lignes extraites avec 55 colonnes.
✗ Erreur lors de l'export : [Errno 21] Is a directory: '/home/bapt/code/Monitor-the-Reactor/raw_data/data_2h'


,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0,1,1,0.25171,3672.4,4466.3,9.5122,27.057,42.473,2705.6,...,54.494,24.527,59.710,22.357,40.149,40.074,47.955,47.300,42.100,15.345
1,0,1,2,0.25234,3642.2,4568.7,9.4145,26.999,42.586,2705.2,...,53.269,24.465,60.466,22.413,39.956,36.651,45.038,47.502,40.553,16.063
2,0,1,3,0.24840,3643.1,4507.5,9.2901,26.927,42.278,2703.5,...,54.000,24.860,60.642,22.199,40.074,41.868,44.553,47.479,41.341,20.452
3,0,1,4,0.25153,3628.3,4519.3,9.3347,26.999,42.330,2703.9,...,53.860,24.553,61.908,21.981,40.141,40.066,48.048,47.440,40.780,17.123
4,0,1,5,0.21763,3655.8,4571.0,9.3087,26.901,42.402,2707.7,...,53.307,21.775,61.891,22.412,37.696,38.295,44.678,47.530,41.089,18.681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,0,417,677,0.22912,3675.4,4429.6,9.4759,27.052,42.193,2704.4,...,53.971,22.363,60.634,22.457,41.068,37.968,47.125,45.816,41.326,19.678
201,0,417,678,0.22899,3665.7,4510.6,9.3188,27.035,42.607,2704.8,...,53.001,22.375,59.772,22.477,41.085,38.568,46.181,45.887,40.728,21.490
202,0,417,679,0.25641,3671.9,4490.9,9.4537,26.870,42.590,2709.1,...,54.299,25.024,62.499,22.653,40.733,38.880,47.604,45.560,40.890,16.500
203,0,417,680,0.25409,3678.0,4533.2,9.4471,26.626,42.491,2712.3,...,53.793,25.253,61.464,22.747,41.043,32.892,44.408,45.648,41.622,17.987
